# Day 1 - Examples: Code Generation

In this notebook we offer an example of code generation. That is, we use the model to help us in a programming task. Note that we use the text completion APIs here as we do not need to have a conversation with the model. We only give it instructions and expect an answer.

We first start with generating SQL based on a given input table. Later in this notebook we also offer some examples for Python generation.

References/Further Reading:

* 

In [1]:
# Load environment variables
from dotenv import load_dotenv

load_dotenv("../../.env")

True

# SQL Agent

## Setup template

We setup a template here asking the model to look at a table schema and answer a set of questions. Feel free to customize the questions and schema to your own liking!

In [12]:
table = '<table>'
template_prompt=f'''Given the following SQL table and set of questions, return the queries corresponding to each question. \nTable: \"\"\"{table}\"\"\"\n \nQuestions:\n0. How many employees are in the sales department?\n1. Which employee earns the most?\n2. Which is the biggest department?'''
print(template_prompt)

Given the following SQL table and set of questions, return the queries corresponding to each question. 
Table: """<table>"""
 
Questions:
0. How many employees are in the sales department?
1. Which employee earns the most?
2. Which is the biggest department?


#### User Input

Here we add in the document we want the model to read. Feel free to modify this to a document of your choice!

In [13]:
select_table = """
Employee(id, name, department_id)
Department(id, name, address)
Salary(employee_id, amount)
"""

## OpenAI

Summarizing text with OpenAI models is pretty simple. We pretty much just send the template (with our document added in) to the Completion API and the output will be the model's attempt at a summary!

In [17]:
import openai

prompt = template_prompt.replace('<table>', select_table)

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt},
]

chatbot_response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=messages,
  temperature=0,
  max_tokens=1500,
)

print(chatbot_response.choices[0].message["content"])

0. How many employees are in the sales department?
```sql
SELECT COUNT(*) 
FROM Employee 
JOIN Department ON Employee.department_id = Department.id 
WHERE Department.name = 'sales';
```

1. Which employee earns the most?
```sql
SELECT Employee.id, Employee.name 
FROM Employee 
JOIN Salary ON Employee.id = Salary.employee_id 
WHERE Salary.amount = (SELECT MAX(amount) FROM Salary);
```

2. Which is the biggest department?
```sql
SELECT Department.id, Department.name, COUNT(Employee.id) AS employee_count 
FROM Department 
JOIN Employee ON Department.id = Employee.department_id 
GROUP BY Department.id, Department.name 
ORDER BY employee_count DESC 
LIMIT 1;
```


## Google

Summarizing text with PaLM is also trivial. We more or less do the same thing as we did for OpenAI and send the prompt to the model.

In [15]:
from vertexai.preview.language_models import TextGenerationModel

prompt = template_prompt.replace('<table>', select_table)

model = TextGenerationModel.from_pretrained("text-bison@001")
response = model.predict(prompt, max_output_tokens=1024)
print(response.text)

0. ```
SELECT COUNT(*)
FROM Employee
WHERE department_id = (
  SELECT id
  FROM Department
  WHERE name = 'Sales'
)
```

1. ```
SELECT name
FROM Employee
JOIN Salary
ON Employee.id = Salary.employee_id
ORDER BY amount DESC
LIMIT 1
```

2. ```
SELECT name
FROM Department
ORDER BY COUNT(*) DESC
LIMIT 1
```


Note how the two APIs offer slightly different results. Try playing around with the prompt and see how the results change!

# Python Agent

Getting code from an LLM is as simple as asking for it!

## OpenAI

In [18]:
import openai

prompt = "Write a python script to generate prime numbers."

messages = [
    {"role": "system", "content": "You are an agent designed to write python code to answer questions."},
    {"role": "user", "content": prompt},
]

chatbot_response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=messages,
  temperature=0,
  max_tokens=1500,
)

print(chatbot_response.choices[0].message["content"])

Here's a Python script that generates prime numbers using the Sieve of Eratosthenes algorithm:

```python
def generate_primes(limit):
    primes = []
    sieve = [True] * (limit + 1)
    sieve[0] = False
    sieve[1] = False

    for current_number in range(2, limit + 1):
        if sieve[current_number]:
            primes.append(current_number)
            for multiple in range(current_number * 2, limit + 1, current_number):
                sieve[multiple] = False

    return primes


if __name__ == "__main__":
    limit = int(input("Enter the limit to generate prime numbers up to: "))
    primes = generate_primes(limit)
    print(f"Prime numbers up to {limit}: {primes}")
```

You can run this script, enter a limit, and it will generate all prime numbers up to that limit.


## Google PaLM

In [19]:
from vertexai.preview.language_models import TextGenerationModel

prompt = "Write a python script to generate prime numbers."

model = TextGenerationModel.from_pretrained("text-bison@001")
response = model.predict(prompt, max_output_tokens=1024)
print(response.text)

```python
# This Python script generates prime numbers.

# Import the math library.
import math

# Define a function to check if a number is prime.
def is_prime(n):
    # Check if the number is 1.
    if n == 1:
        return False

    # Check if the number is 2.
    if n == 2:
        return True

    # Check if the number is divisible by any number from 2 to the square root of the number.
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            return False

    # The number is prime if it is not divisible by any number from 2 to the square root of the number.
    return True

# Get the user input.
n = int(input("Enter a number: "))

# Generate a list of prime numbers from 2 to n.
prime_numbers = []
for i in range(2, n + 1):
    if is_prime(i):
        prime_numbers.append(i)

# Print the list of prime numbers.
print("The prime numbers from 2 to {} are:".format(n))
for prime in prime_numbers:
    print(prime)
```
